# Just train and save a resnet18 on CIFAR10 with pytorch lightning

In [8]:
%matplotlib inline

In [9]:
import torch
import torchvision
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
import torch.nn.functional as F
import torch.nn as nn
from torchvision.datasets import CIFAR10

import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers

import torchmetrics
from tqdm.notebook import tqdm
import lightly

# DATA

In [10]:
# data params
num_workers = 6
batch_size = 512

In [11]:
# The dataset structure should be like this:
# cifar10/train/
#  L airplane/
#    L 10008_airplane.png
#    L ...
#  L automobile/
#  L bird/
#  L cat/
#  L deer/
#  L dog/
#  L frog/
#  L horse/
#  L ship/
#  L truck/
path_to_train = './data/cifar10_lightly/train/'
path_to_test = './data/cifar10_lightly/test/'

In [12]:
# ------------- transforms ------------------- #

# Augmentations typically used to train on cifar-10
train_classifier_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor(),
    #     torchvision.transforms.Normalize(
    #         mean=(0.5, 0.5, 0.5),
    #         std=(0.5, 0.5, 0.5),
    #     ),
])

# No additional augmentations for the test set
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),
    torchvision.transforms.ToTensor(),
    #     torchvision.transforms.Normalize(
    #         mean=lightly.data.collate.imagenet_normalize['mean'],
    #         std=lightly.data.collate.imagenet_normalize['std'],
    #     ),
])

# --------------- datasets --------------------- #
dataset_train_classifier = lightly.data.LightlyDataset(
    input_dir=path_to_train,
    transform=train_classifier_transforms
)

dataset_test = lightly.data.LightlyDataset(
    input_dir=path_to_test,
    transform=test_transforms
)

# ------------------ dataloaders ----------------- #
train_dataloader = torch.utils.data.DataLoader(
    dataset_train_classifier,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

val_dataloader = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

# Model

In [13]:
# model hyperparams
max_epochs = 200

In [14]:
from plr18_RESNETCOPY import plr18
model = plr18()

In [22]:
list(model.model.children())[:-2]

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (shortcut): Sequential()
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (shortcut)

# Train

In [9]:
# # Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Reusing TensorBoard on port 6006 (pid 236117), started 8 days, 22:25:03 ago. (Use '!kill 236117' to kill it.)

In [10]:
# you can also define a checkpoint callback to save best model like keras.
checkpoint_callback = ModelCheckpoint(
    dirpath='./saved_models/resnet_80',
    filename='{epoch}-{val_loss:.2f}-{val_acc:.2f}',
    save_top_k=5,
    verbose=True,
    monitor='avg_val_acc',
    mode='max'
)

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ./saved_models/resnet_80 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [11]:
tb_logger = pl_loggers.TensorBoardLogger(save_dir='./lightning_logs/', name='resnet_80')
trainer = Trainer(gpus=1, callbacks=[checkpoint_callback], max_epochs=80, logger=tb_logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [12]:
trainer.fit(model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type             | Params
----------------------------------------------
0 | model    | ResNet           | 11.2 M
1 | criteria | CrossEntropyLoss | 0     
----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.716    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 96: avg_val_acc reached 0.34086 (best 0.34086), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_80/epoch=0-val_loss=1.83-val_acc=0.35.ckpt" as top 5


...

Epoch 77, global step 7565: avg_val_acc reached 0.89686 (best 0.90432), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_80/epoch=77-val_loss=0.47-val_acc=0.90.ckpt" as top 5


Validating: 0it [00:00, ?it/s]

Epoch 78, global step 7662: avg_val_acc reached 0.89828 (best 0.90432), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_80/epoch=78-val_loss=0.48-val_acc=0.90.ckpt" as top 5


Validating: 0it [00:00, ?it/s]

Epoch 79, global step 7759: avg_val_acc was not in top 5
